# Shor Algorithm:

We follow the steps and gates names as in "A General Implementation of Shor’s Algorithm":

- https://medium.com/mit-6-s089-intro-to-quantum-computing/a-general-implementation-of-shors-algorithm-da1595694430

In [1]:
import os
from dotenv import load_dotenv
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math

In [2]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("QUANTUM_RINGS_TOKEN")
name = os.getenv("QUANTUM_RINGS_NAME")

provider = QuantumRingsProvider(
    token=token,
    name=name
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

{'name': 'cffuidio@gmail.com',
 'token': 'rings-200.t2iD6huiMZ4DMUowJyfZFMiW3cGGrqF6',
 'max_qubits': '200'}

In [68]:
def iqft_cct(qc, b, qubit_list):
    """
    The inverse QFT circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        n (int):
                The number of qubits in the registers to use

    Returns:
        None

    """

    n = len(qubit_list)

    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            qc.cu1(  -math.pi / 2** ( i -j + 1 ), b[qubit_list[j - 1]], b[qubit_list[i]])
        # the H transform should be done after the rotations
        qc.h(b[qubit_list[i]])
    qc.barrier()
    return



def qft_cct(qc, b, qubit_list):
    """
    The QFT circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        n (int):
                The number of qubits in the registers to use

    Returns:
        None

    """

    n = len(qubit_list)

    for i in range (n):
    
        qc.h(b[qubit_list[i]])
        for j in range (i+1, n):
            qc.cu1( math.pi / 2**(i - j + 1), b[qubit_list[j]], b[qubit_list[i]]  )


    qc.barrier()

    for i in range(int(n/2)):
        qc.swap(b[qubit_list[i]], b[qubit_list[(n-1)-i]])

    qc.barrier()


    return


def adder_gate(qc, b, qubit_list, a):
    """
    The adder circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

    Returns:
        None

    """


    n = len(qubit_list)

    for i in range(n):
        exponent = (n - i)
        qc.rz( 2 * math.pi * a / (2**(exponent)), b[qubit_list[i]])

    qc.barrier()

    return


def substract_gate(qc, b, qubit_list, a):

    """
    The inverse of adder circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

    Returns:
        None

    """

    # Inverse of adder gate:

    n = len(qubit_list)

    for i in range(n):
        exponent = (n - i)
        qc.rz( - 2 * math.pi * a / (2**(exponent)), b[qubit_list[i]]) # Only change sign

    qc.barrier()

    return


def control_adder_gate(qc, b, qubit_list, a, control):

    """
    Control adder gate

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

        control (int):
                Control qubit number

    Returns:
        None

    """

    n = len(qubit_list)

    for i in range(n):
        exponent = (n - i)
        qc.crz( 2 * math.pi * a / (2**(exponent)), control, b[qubit_list[i]])

    qc.barrier()

    return




def modular_adder(qc, b, qubit_list, a, N, ancilla_QFT, ancilla):

    """
    Modular adder gate

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

        N (int):
                Parameter representing the mod (mod N)

        ancilla_QFT (int):
                Number of the ancilla qubit used for QFT (see paper)

        ancilla (int):
                Number of the ancilla qubit

    Returns:
        None

    """

    qft_list = qubit_list
    qft_list.append(ancilla_QFT)

    adder_gate(qc, b, qubit_list, a)
    substract_gate(qc, b, qubit_list, N)

    iqft_cct(qc, b, qft_list) # Use QFT to
    qc.cx(ancilla_QFT, ancilla)
    qft_cct(qc, b, qft_list)

    control_adder_gate(qc, b, qubit_list, N, ancilla)
    substract_gate(qc, b, qubit_list, a)


    iqft_cct(qc, b, qft_list) # Use QFT to
    qc.x(ancilla_QFT)
    qc.cx(ancilla_QFT, ancilla)
    qc.x(ancilla_QFT)
    qft_cct(qc, b, qft_list)


    adder_gate(qc, b, qubit_list, a)

    return


# Finish this
def control_modular_adder(qc, b, qubit_list, a, N, ancilla_QFT, ancilla, control):

    """
    Control Modular adder gate

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

        N (int):
                Parameter representing the mod (mod N)

        ancilla_QFT (int):
                Number of the ancilla qubit used for QFT

        ancilla (int):
                Number of the ancilla qubit

        control (int):
                Number of control qubit

    Returns:
        None

    """



    return


def multiplier_gate(qc, b, qubit_list, a, N, x_qubit_list, ancilla_QFT, ancilla): # We have to do the control multiplier and the control swap.
    # For finish the control multiplier we have to finish the control_modular_adder

    """
    Multiplier gate

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        qubit_list (int[]):
                number of qubits to use

        a (float):
                Parameter

        N (int):
                Parameter representing the mod (mod N)

        x_qubit_list (int[])
                List of qubits in x register

        ancilla_QFT (int):
                Number of the ancilla qubit used for QFT

        ancilla (int):
                Number of the ancilla qubit

        control (int):
                Number of control qubit

    Returns:
        None

    """

    n = len(qubit_list)

    qft_cct(qc, b, qubit_list)


    for i in range(n):
        control_modular_adder(qc, b, qubit_list, a * 2**i, N, ancilla_QFT, ancilla, x_qubit_list[i])

    qft_cct(qc, b, qubit_list)

    return

In [69]:
# Shor’s algorithm to factorize 15 using 7^x mod 15.
numberofqubits = 7
shots = 1024

q = QuantumRegister(numberofqubits , 'q')
c = ClassicalRegister(3 , 'c')
qc = QuantumCircuit(q, c)


modular_adder(qc, q, [i for i in range(4, 7)], 2, 5, 1, 2)

# Draw the circuit
qc.draw("mpl")

                        ╎                    ╎                                         »
q[0]: ■─────────────────╎────────────────────╎─────────────────────────────────────────»
       ┌──────────────┐ ╎ ┌────────────────┐ ╎                                         »
q[1]: ■┤ RZ(6.283185) ├─╎─┤ RZ(-15.707963) ├─╎─────────────────────────────────────────»
       └──────────────┘ ╎ └────────────────┘ ╎                                         »
q[2]: ■─────────────────╎────────────────────╎─────────────────────────────────────────»
                        ╎                    ╎                                         »
q[3]: ■─────────────────╎────────────────────╎─────────────────────────────────────────»
       ┌──────────┐     ╎ ┌────────────────┐ ╎ ┌───┐                                   »
q[4]: ■┤ RZ(pi/4) ├─────╎─┤ RZ(-1.963495)  ├─╎─┤ H ├───────■───────────────────■───────»
       ├──────────┤     ╎ ├────────────────┤ ╎ └───┘┌──────┴──────┐┌───┐       │       »
q[5]: ■┤ RZ(pi/2) ├──